## Slidebar annimated Geo Visual of rental prices (2013-2022)
This notebook visualises the averaged rental prices for all properties in our dataset from 2013 - 2022. The generated geo visual plot with a slide bar will easily demonstrate the trend of overall rental price value from 2013 - 2022

**results**
1. the overall rental prices increases for most suburbs from 2013 to 2022
2. largest rental prices changes occurred in central vic (melbs) area

**implication**

rental price may be affected due to economic factors such as inflation. Hence we could include GDP, saving rate, income, population as economic factors which may influence rental price.

In [ ]:
import pandas as pd
import glob
import os

## Analyse Combined Data

In [ ]:
curated_path = "../data/curated/merged_dataset/*.csv"
file_lst2 = []
for fname in glob.glob(curated_path):
    file_lst2.append(fname)
file_lst2
# combine all files in the list
merged_df = pd.concat([pd.read_csv(f) for f in file_lst2 ])
merged_df.dropna(inplace=True)
merged_df = merged_df[['year', 'sa2_2021', 'weekly_rent']]
merged_df = merged_df.groupby(['year', 'sa2_2021'], as_index=False).agg({'weekly_rent': 'mean'})

In [ ]:
vic_sa2 = pd.read_csv('../data/curated/sa2_vic_2021.csv').iloc[:, 1:]
vic_sa2.rename({'SA2_CODE21': 'sa2_2021', 'SA2_NAME21': 'name_2021'}, axis=1, inplace=True)

In [ ]:
all_year_df = merged_df.merge(vic_sa2, how='left', left_on='sa2_2021', right_on='sa2_2021')
all_year_df.head()

## Geo Anlaysis

In [ ]:
import geopandas as gpd
import json
import pandas as pd
with open('../data/curated/geo.json', 'r') as filename:
    geoJSON=json.load(filename)
geoJSON[:500]

In [ ]:
#Preparing the geometries for neighbourhoods
neighbourhoods = gpd.read_file(geoJSON)

#The index of the json has to be the neighbourhood name
neighbourhoods.index = neighbourhoods['SA2_NAME21']
#Dropping useless columns for this application (it's all about saving memory)
neighbourhoods.drop(['SA2_NAME21','id'], axis=1, inplace=True)

#Choropleth mapbox accepts a json for the geometries of neighbourhoods.
neighbourhoods_json = json.loads(neighbourhoods.to_json())
neighbourhoods.head()

In [ ]:
all_year_df = merged_df.merge(vic_sa2, how='left', left_on='sa2_2021', right_on='sa2_2021')


In [ ]:
all_year_df.head()

In [ ]:

import folium
m = folium.Map(location=[-37.81, 144.96], tiles="Stamen Terrain", zoom_start=10, color='white')
svg_style = '<style>svg {background-color: rgb(255, 255, 255,0.5);}</style>'
m.get_root().header.add_child(folium.Element(svg_style))

c = folium.Choropleth(
    geo_data=geoJSON,
    name='choropleth',
    data=all_year_df[all_year_df['year']==2022], 
    columns=['name_2021','weekly_rent'],
    key_on='properties.SA2_NAME21', 
    fill_color='PiYG', 
    nan_fill_color='black',
    legend_name='weekly rent by SA2',
)

c.add_to(m)
m

In [ ]:
max_count = all_year_df['weekly_rent'].max()

import plotly.express as px
fig = px.choropleth_mapbox(data_frame=all_year_df,
                           geojson=neighbourhoods,
                           locations=all_year_df['name_2021'],
                           color='weekly_rent',
                           center={'lat':-37.81, 'lon':144.96},
                           mapbox_style='open-street-map',
                           zoom=5,
                           color_continuous_scale='blues',
                           range_color=(0, 1000),
                           animation_frame='year',
                           width=800,
                           height=600)
fig.write_html('../plots/aggregated_geo/plow_map.html')
#fig.show()